### Load tensorflow

In [0]:
import tensorflow as tf

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
#@title Default title text
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd
project_path = "/content/drive/My Drive/Colab Notebooks/"

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prices.csv')

### Check all columns in the dataset

In [35]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [38]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = data.iloc[:1000,:]


### Convert Float64 to Float32

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float32
close     1000 non-null float32
low       1000 non-null float32
high      1000 non-null float32
volume    1000 non-null float32
dtypes: float32(5)
memory usage: 19.6 KB


In [0]:
data = data.astype('float32')

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
open      1000 non-null float64
close     1000 non-null float64
low       1000 non-null float64
high      1000 non-null float64
volume    1000 non-null float64
dtypes: float64(5)
memory usage: 39.1 KB


### Divide the data into train and test sets

In [0]:
X = data.drop(columns='close')
y = data['close']


In [0]:
#import sklean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

### Normalize Train and Test Data 

In [0]:
norm = Normalizer()
X_train = norm.fit_transform(X_train)
X_test = norm.fit_transform(X_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
# y = wx + b

w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [72]:
type(w)

tensorflow.python.framework.ops.EagerTensor

3.Prediction

In [0]:
def prediction(x,w,b):
  y_pred = tf.add(tf.matmul(x,w),b)
  return(y_pred)

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg
  

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x,y,w,b,learning_rate=0.01):
  with tf.GradientTape() as t:
    t.watch([w,b])
    current_prediction = prediction(x, w, b)
    current_loss = loss(y, current_prediction)
    dw, db = t.gradient(current_loss,[w, b])
    
    w = w - learning_rate*dw
    b = b - learning_rate*db
    return w, b

## Execute the Graph for 100 epochs and observe the loss

In [104]:
import numpy as np
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train, np.array(y_train), w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, 
          loss(np.array(y_train), prediction(X_train, w, b)).numpy())

Current Loss on iteration 0 3358.092
Current Loss on iteration 1 3358.005
Current Loss on iteration 2 3357.923
Current Loss on iteration 3 3357.8503
Current Loss on iteration 4 3357.7827
Current Loss on iteration 5 3357.72
Current Loss on iteration 6 3357.6616
Current Loss on iteration 7 3357.609
Current Loss on iteration 8 3357.559
Current Loss on iteration 9 3357.5137
Current Loss on iteration 10 3357.4736
Current Loss on iteration 11 3357.4353
Current Loss on iteration 12 3357.3984
Current Loss on iteration 13 3357.3672
Current Loss on iteration 14 3357.3352
Current Loss on iteration 15 3357.309
Current Loss on iteration 16 3357.2832
Current Loss on iteration 17 3357.26
Current Loss on iteration 18 3357.2393
Current Loss on iteration 19 3357.2185
Current Loss on iteration 20 3357.2004
Current Loss on iteration 21 3357.1816
Current Loss on iteration 22 3357.1672
Current Loss on iteration 23 3357.1528
Current Loss on iteration 24 3357.1377
Current Loss on iteration 25 3357.1272
Curren

### Get the shapes and values of W and b

In [106]:
w,b

(<tf.Tensor: id=5182668, shape=(4, 1), dtype=float32, numpy=
 array([[2.5363823e-03],
        [2.5116093e-03],
        [2.5585014e-03],
        [3.3847992e+01]], dtype=float32)>,
 <tf.Tensor: id=5182671, shape=(1,), dtype=float32, numpy=array([33.847992], dtype=float32)>)

In [107]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5363823e-03]
 [2.5116093e-03]
 [2.5585014e-03]
 [3.3847992e+01]]
Bias:
 [33.847992]


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [108]:
import keras

Using TensorFlow backend.


In [0]:
model = tf.keras.Sequential()


In [0]:
model.add(tf.keras.layers.Dense(1,input_shape=(4,)))


In [0]:
model.compile(optimizer='sgd',loss = 'mse')

### Execute the model

In [123]:
model.fit(X_train,np.array(y_train),epochs=100)

Epoch 1/100
800/800 [==============================] - 0s 32us/sample - loss: 3360.1771
Epoch 2/100
800/800 [==============================] - 0s 31us/sample - loss: 3358.5961
Epoch 3/100
800/800 [==============================] - 0s 29us/sample - loss: 3360.7633
Epoch 4/100
800/800 [==============================] - 0s 28us/sample - loss: 3359.9872
Epoch 5/100
800/800 [==============================] - 0s 29us/sample - loss: 3362.2919
Epoch 6/100
800/800 [==============================] - 0s 28us/sample - loss: 3359.2946
Epoch 7/100
800/800 [==============================] - 0s 27us/sample - loss: 3359.9904
Epoch 8/100
800/800 [==============================] - 0s 27us/sample - loss: 3360.2129
Epoch 9/100
800/800 [==============================] - 0s 29us/sample - loss: 3359.7333
Epoch 10/100
800/800 [==============================] - 0s 26us/sample - loss: 3360.3870
Epoch 11/100
800/800 [==============================] - 0s 27us/sample - loss: 3359.8268
Epoch 12/100
800/800 [========

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Iris.csv')

### Splitting the data into feature set and target set

In [0]:
data.set_index('Id',inplace=True)

In [0]:
X = data.drop(columns='Species')
y = data.Species

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
y = pd.get_dummies(y)

In [164]:
np.array(y)

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

### Divide the dataset into Training and test (70:30)

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model2 = tf.keras.Sequential()

In [0]:
model2.add(tf.keras.layers.BatchNormalization())

In [0]:
model2.add(tf.keras.layers.Dense(10,input_shape=(4,)))

In [0]:
model2.add(tf.keras.layers.Dense(8,))

In [0]:
model2.add(tf.keras.layers.Dense(3,activation='softmax'))

### Fitting the model and predicting 

In [0]:
model2.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])


In [167]:
model2.fit(np.array(X_train),np.array(y_train),epochs=100,batch_size=10)

Epoch 1/100
120/120 [==============================] - 0s 243us/sample - loss: 0.4202 - acc: 0.8083
Epoch 2/100
120/120 [==============================] - 0s 139us/sample - loss: 0.3753 - acc: 0.8333
Epoch 3/100
120/120 [==============================] - 0s 129us/sample - loss: 0.4021 - acc: 0.8167
Epoch 4/100
120/120 [==============================] - 0s 135us/sample - loss: 0.3717 - acc: 0.8417
Epoch 5/100
120/120 [==============================] - 0s 137us/sample - loss: 0.3483 - acc: 0.8583
Epoch 6/100
120/120 [==============================] - 0s 135us/sample - loss: 0.3304 - acc: 0.8833
Epoch 7/100
120/120 [==============================] - 0s 130us/sample - loss: 0.3597 - acc: 0.8667
Epoch 8/100
120/120 [==============================] - 0s 132us/sample - loss: 0.3684 - acc: 0.8333
Epoch 9/100
120/120 [==============================] - 0s 135us/sample - loss: 0.3480 - acc: 0.8417
Epoch 10/100
120/120 [==============================] - 0s 141us/sample - loss: 0.3985 - acc: 0.8250

In [0]:
y_pred = model2.predict(X_test)

In [169]:
y_pred

array([[4.03917838e-05, 6.58812404e-01, 3.41147304e-01],
       [2.14183889e-02, 8.73615503e-01, 1.04966134e-01],
       [1.32985428e-04, 2.12782770e-01, 7.87084281e-01],
       [2.59923493e-03, 4.08329397e-01, 5.89071393e-01],
       [6.12240471e-03, 4.52223957e-01, 5.41653633e-01],
       [9.99530911e-01, 4.69009101e-04, 9.09750284e-08],
       [1.61598379e-07, 3.00458651e-02, 9.69953954e-01],
       [5.63511252e-03, 9.22987401e-01, 7.13775232e-02],
       [4.24088724e-03, 7.58977413e-01, 2.36781731e-01],
       [2.57749070e-06, 1.55147508e-01, 8.44849944e-01],
       [9.98309016e-01, 1.68845837e-03, 2.53063649e-06],
       [9.99968648e-01, 3.13555902e-05, 6.25349017e-09],
       [9.98329930e-03, 3.76142740e-01, 6.13873899e-01],
       [3.41003342e-03, 8.51447344e-01, 1.45142645e-01],
       [3.09102912e-03, 9.56472456e-01, 4.04365472e-02],
       [5.47135369e-06, 9.75519493e-02, 9.02442575e-01],
       [2.10473780e-03, 8.08845520e-01, 1.89049840e-01],
       [9.98931706e-01, 1.06816

### Report Accuracy of the predicted values

In [181]:
model2.evaluate(np.array(X_train),np.array(y_train)) #train accuracy

120/120 [==============================] - 0s 554us/sample - loss: 0.1386 - acc: 0.9667


[0.13857474128405253, 0.96666664]

In [182]:
model2.evaluate(np.array(X_test),np.array(y_test)) #test accuracy

30/30 [==============================] - 0s 1ms/sample - loss: 0.2132 - acc: 0.8667


[0.21316629648208618, 0.8666667]